<a href="https://colab.research.google.com/github/sankardevisharath/amex-default-prediction/blob/master/notebooks/Separate_Static_Dynamic_Features_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from os import listdir

files = listdir('/content/drive/MyDrive/amex-default-prediction/data/customer_train/')


In [8]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [9]:
pip install tensorflow_decision_forests


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.0 MB 5.2 MB/s 
     |████████████████████████████████| 511.7 MB 5.5 kB/s 
     |████████████████████████████████| 1.6 MB 51.9 MB/s 
     |████████████████████████████████| 438 kB 74.8 MB/s 
     |████████████████████████████████| 5.8 MB 34.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Unins

In [2]:
import gc 

import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import tensorflow as tf
from keras.layers import GRU, Dense
import tensorflow_addons as tfa
import tensorflow_decision_forests as tfdf



In [3]:
cat_cols = ['D_63', 'D_64','B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_66', 'D_68']
TRAIN_LABELS_PATH = '/content/train_labels.csv'
AGG_DATA_ABS_PATH = '/content/drive/MyDrive/amex-default-prediction/data/aggr/customer-agg.parquet'

In [4]:
!unzip -o /content/drive/MyDrive/amex-default-prediction/data/amex-default-prediction.zip \
 train_labels.csv

Archive:  /content/drive/MyDrive/amex-default-prediction/data/amex-default-prediction.zip
  inflating: train_labels.csv        


In [5]:
train_labels = pd.read_csv(TRAIN_LABELS_PATH)

In [6]:
train_labels.shape

(458913, 2)

In [ ]:
agg_dataset = pd.read_parquet(path=AGG_DATA_ABS_PATH)

In [ ]:
enc_agg = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
agg_dataset[['D_63_last', 'D_64_last']]=enc_agg.fit(agg_dataset[['D_63_last', 'D_64_last']])
agg_dataset[['D_63_last', 'D_64_last']]=enc_agg.transform(agg_dataset[['D_63_last', 'D_64_last']])
agg_dataset.fillna(0, inplace=True)



In [ ]:
agg_dataset.drop(axis=1, columns=['target'], inplace=True)

In [ ]:
agg_dataset.shape

(458913, 919)

In [8]:
def append_label(source_df):
  return pd.merge(left=source_df, right=train_labels, how='inner')

In [29]:
dynamic_col = []
X = pd.DataFrame()
Y = pd.DataFrame()
df_tmp = pd.read_parquet('/content/drive/MyDrive/amex-default-prediction/data/customer_train/0_50000.parquet')
df_tmp = append_label(df_tmp)


X= df_tmp

In [30]:
X = X[X.groupby('customer_ID')['customer_ID'].transform('count') == 13]
X= X.sort_values('customer_ID')

In [37]:
X['D_137'].unique()

array([       nan, 0.00471206, 0.00354082, ..., 0.00324144, 0.00015806,
       0.00098207])

In [41]:
Q = X[X['customer_ID'] =='0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a']

In [54]:
D_68_Test = X[X['customer_ID'] =='000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc']

In [55]:
D_68_Test['D_68']

50    3.0
49    3.0
48    3.0
47    3.0
46    3.0
45    3.0
51    3.0
43    3.0
42    6.0
41    6.0
40    2.0
39    2.0
44    3.0
Name: D_68, dtype: float64

In [49]:
Q['D_68']

0     6.0
12    6.0
11    6.0
10    6.0
8     6.0
7     6.0
9     6.0
5     6.0
4     6.0
3     6.0
2     6.0
1     6.0
6     6.0
Name: D_68, dtype: float64

In [46]:
R = Q.nunique()

In [48]:
R[R<=1]

customer_ID    1
D_42           0
D_43           0
D_49           0
D_53           0
D_63           1
D_64           1
B_17           0
D_66           0
D_68           1
D_73           0
D_76           0
R_9            0
B_29           0
B_30           1
D_87           0
D_88           0
B_31           1
D_106          0
R_26           0
B_38           1
D_108          0
D_110          0
D_111          0
B_39           0
D_114          1
D_116          1
D_117          1
D_120          1
D_126          1
B_42           0
D_132          0
D_134          0
D_135          0
D_136          0
D_137          0
D_138          0
D_142          0
target         1
dtype: int64

In [36]:
X['D_87'].unique()

array([nan,  1.])

In [31]:
P = X.groupby('customer_ID').nunique()

In [32]:
P['D_137'].unique()

array([ 0,  5,  7, 10,  8,  6,  4,  3,  9,  2,  1, 11, 12, 13])

In [33]:
T =  X.groupby('customer_ID').nunique() <=1

In [34]:
T

,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,False,False,False,False,False,False,False,False,False,True,...,True,True,False,False,False,True,False,False,False,True
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,False,False,False,False,False,False,False,False,False,True,...,True,True,False,False,False,True,False,False,False,True
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,False,False,False,False,False,False,True,False,False,True,...,True,True,False,False,False,True,False,False,False,True
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,False,False,False,False,False,False,False,False,False,True,...,True,True,False,False,False,True,False,False,False,True
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,False,False,False,False,False,False,False,False,False,True,...,True,True,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1c065659d2a797ab2519fb07dd53cf332d11f01ef1ab00b4795a61ca73df91b6,False,False,False,False,False,False,False,False,False,True,...,True,True,False,False,False,False,False,False,False,True
1c06b7ba5a082b12f31a28396f8bc8675e3fc218544577ce98283e8a7972a6fc,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,True
1c06e529cbe7b3d281ee4a6b0aa3484f67f1de45de1b5a4b45b3bf4e8a01a069,False,False,False,False,False,False,False,False,False,True,...,True,True,False,False,False,False,False,False,False,True


In [35]:
T.columns[T.eq(True).all()]

Index(['D_87', 'target'], dtype='object')

In [53]:
T[T['D_68'] == False]

,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
customer_ID,,,,,,,,,,,,,,,,,,,,,
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,False,False,False,False,False,False,False,False,False,True,...,True,True,False,False,False,True,False,False,False,True
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f5235f98b0f47c9d7d8d4,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,True
0000f99513770170a1aba690daeeb8a96da4a39f11fc27da5c30a79db61c1e85,False,False,False,False,False,False,False,False,False,True,...,True,True,False,False,False,False,False,False,False,True
00013c6e1cec7c21bede7cb319f1e28eb994f5625257f479c53ad6e90c177f7c,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,True,False,False,False,True
0001812036f1558332e5c0880ecbad70b13a6f28ab04a8db6d83a26ef40aadb0,False,False,False,False,False,False,False,False,False,True,...,True,True,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1c047d86a4e7f0a456f52cb70494ae4d7ae7dfb20ebbf033c6f8f704f30714cc,False,False,False,False,False,False,False,False,False,True,...,True,True,False,False,False,True,False,False,False,True
1c04841edde899dc8dd5f27aaada0489b23c1c58346b9812990fd4fd7983a4ea,False,False,False,False,False,False,True,False,False,True,...,True,True,False,False,False,False,False,False,False,True
1c0616f7f4824c29aa7c3070b4c9dd9ada399fdf41a94d308332a32cd8547a63,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
